In [ ]:
from sequence.protocol import Protocol
from sequence.message import Message
from enum import Enum, auto

# Estados possíveis do protocolo de entrelaçamento com handshake
class HandshakeState(Enum):
    CLOSED = auto()
    SYN_SENT = auto()
    SYN_RECEIVED = auto()
    ESTABLISHED = auto()
    FINESHED = auto()
    FAILED = auto()

# Tipos de mensagem do protocolo de entrelaçamento
class HandshakeMessageType(Enum):
    SYN = auto()
    SYN_ACK = auto()
    ACK = auto()
    
# Mensagem de hanshake do protocolo de entrelaçamento
class EntangleConnectionHandshakeMessage(Message):
    """
    A mensagem é composta por um message_type (SYN, SYN_ACK, ACK), um session_id (id da sessão/tentativa de entrelaçamento) e um memory_id (id da memória que vai ser usada).
    """
    # Protocol_receiver é o protocolo que vai receber a mensagem no remote_node, quando o owner envia a mensagem.
    # Ele identifica por if protocol.name == msg.receiver
    # o memory_id é o id da memória que o owner do protocolo vai usar para o entrelaçamento
    def __init__(self, msg_type, session_id, protocol_receiver, memory_id=None):
        super().__init__(msg_type, protocol_receiver)
        self.session_id = session_id
        self.memory_id = memory_id

    def __str__(self):
        return f"{self.msg_type.name} | session: {self.session_id} | mem: {self.memory_id}"

class EntangleConectionProtocol(Protocol):
    """
    Protocolo de entrelaçamento entre dois nós. A e B serão entrelaçados, sendo A o nó que solicita o entrelaçamento e B o nó que responde. O protocolo funciona com um handshake de três etapas: SYN, SYN_ACK e ACK.
    
    """
    # O session_id serve para identificar a sessão de entrelaçamento
    # O name é o nome do protocolo, que é uma string que identifica o protocolo, serve para o roteamento das mensagens
    # O owner é o nó que tem o protocolo, o "dono" da instância, mesmo que seja quem recebeu ou enviou a solicitação de entrelaçamento
    # O remote_node é o nó que vai receber a solicitação de entrelaçamento, mas ele também tem uma instância onde ele é o owner do protocolo
    # O target_memory é a memória que vai ser usada para o entrelaçamento, ela é alocada quando o protocolo é iniciado
    def __init__(self, owner, remote_node):
        self.session_id = f'[{self.owner.name}-{self.remote_node.name}]{id(self)}'
        name = f"ech2-{self.owner.name}-{self.session_id}"
        super().__init__(owner, name)
        self.remote_node = remote_node
        self.target_memory = None
    
    def start(self):
        """
        Inicia o protocolo de entrelaçamento, entre A e B. Sendo A o que solicita o entrelaçamento e B o que responde.
        """
        # Envia a mensagem de SYN para o remote_node
        self.send_message(self.remote_node, HandshakeMessageType.SYN)
        
        # Muda o próprio estado para SYN_SENT
        self.set_state(HandshakeState.SYN_SENT)
    
    def send_message(self, dest, msg_type):
        """
        Envia a mensagem para o remote_node.
        
        Args:
            dest (str): O nó de destino.
            msg_type (Enum): O tipo da mensagem.
        """
        # marca o protocolo que vai receber a mensagem
        receiver_protocol = f"ech{self.remote_node.name}-2-{self.owner.name}"
        
        # cria a mensagem de handshake
        msg = EntangleConnectionHandshakeMessage(
            msg_type=msg_type,
            session_id=self.session_id,
            protocol_receiver=receiver_protocol,
            memory_id=self.target_memory.id if msg_type != HandshakeMessageType.SYN else None
        )
        
        # envia a mensagem para o nó de destino
        self.owner.send_message(dest, msg)
        
        
    def received_message(self, src, msg):
        """
        Recebe a mensagem do remote_node e processa de acordo com o estado do protocolo.
        """
        if msg.msg_type == HandshakeMessageType.SYN:      
            # então sou um B, tem um hub querendo se conectar
            # aloca a memória que vai ser usada para o entrelaçamento
            self.memory = self.owner.allocate_raw_memory()
            
            # se nenhuma memória disponível
            if self.memory is None:
                print("Nenhuma memória disponível")
                return
            
            # responde com um SYN_ACK
            self.send_message(src, HandshakeMessageType.SYN_ACK)
            
            # muda o estado para SYN_RECEIVED
            self.set_state(HandshakeState.SYN_RECEIVED)
            
        elif msg.msg_type == HandshakeMessageType.SYN_ACK:
            # então sou um A, já enviei o SYN e recebi o SYN_ACK
            self.owner.allocate_memory(msg.memory_id)
            
            # se não tiver memória disponível, então encerra o protocolo
            if self.memory is None:
                print("Nenhuma memória disponível")
                return
            
            # recebi qual a memória que o receiver quer usar, então posso enviar a mensagem de ACK com o id da memória que vou usar
            # responde com um ACK
            self.send_message(src, HandshakeMessageType.ACK)
            
            # muda o estado para ESTABLISHED
            self.set_state(HandshakeState.ESTABLISHED)
            
        elif msg.msg_type == HandshakeMessageType.ACK:
            # então sou um B, já enviei o SYN_ACK e recebi o ACK
            
            # finaliza o handshake
            self.set_state(HandshakeState.FINISHED)
        
        else:
            # se a mensagem não for do tipo SYN, SYN_ACK ou ACK, então encerra o protocolo
            print("Mensagem inválida")
            self.set_state(HandshakeState.FAILED)
    
    
    def set_state(self, state):
        # se o estado mudar para FAILED então encerra
        # self.close()
        self.state = state

    def close(self):
        if self.target_memory
    

SyntaxError: expected ':' (599150939.py, line 144)

In [ ]:
from sequence.kernel.timeline import Timeline
from sequence.components.memory import MemoryArray
from sequence.topology.node import Node
from sequence.resource_management.resource_manager import ResourceManager

# cria uma instância de Timeline
runtime = 1e12  # tempo máximo de simulação em picosegundos
tl = Timeline(runtime)

# Hub tem que ter memória, gerenciar os entrelaçamentos
class Hub(Node):
    """Nó responsável por gerenciar a memória e os entrelaçamentos. Fica entre os sensores e o gateway de saída."""
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        # Memória de qubits, são esses argumentos
        memory_array_name = f"{self.name}.memory_array"
        # outros detalhes envolvem parametros de configuração da memória, mas por enquanto só isso
        self.memory_array = MemoryArray(memory_array_name, timeline, num_memories=10)
        # adiciona as memórias ao hub
        self.add_component(self.memory_array)
        # indica que a memória é desse hub
        self.memory_array.add_receiver(self)
        # gerenciar os entrelaçamentos (pode ser melhor definir o próprio, mas por enquanto é assim)
        self.resource_manager = ResourceManager(self, memory_array_name)

    def get(self, photon, **kwargs):
        src = kwargs.get("src")
        dst = kwargs.get("dst")

        # se não for o destino, apenas reencaminha o photon para o destino correto
        if dst != self.name:
            print(f"[{self.name}] qubit de {src} redirecionado para {dst}")
            self.send_qubit(dst, photon)
            return

        # tenta alocar memória RAW
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                # isso é opcional pq é meio que de mentirinha, o quantummanager faz o trabalho de verdade
                info.memory.photon = photon
                print(f"[{self.name}] armazenou qubit de {src} em {info.memory.name}")
                return

        # se não encontrou memória, descarta o fóton
        print(f"[{self.name}] MEMÓRIA CHEIA. Qubit de {src} descartado.")
    
    def alocate_raw_memory(self):
        # TODO: verificar se esse é o mesmo caso do get, pra mim, parece diferente. não sei ainda por que
        memory = self.check_raw_memory()
        if memory is None:
            return 
        # Retorna a memória RAW disponível
        self.resource_manager.update(None, memory, "OCCUPIED")
        return memory
    
    def check_raw_memory(self):
        # verifica se tem memória RAW disponível
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                return info.memory
        
        
hub = Hub("hub", tl)
for m in hub.resource_manager.memory_manager:
    print(type(m))
    print(f"[Hub {hub.name}] memória {m.memory.name} está {m.state}.")
# configuração opcional: desativar barra de progresso
tl.show_progress = False